# jupyter lab --ip=0.0.0.0
# start-master.sh
# start-worker.sh spark://yash-kukrejade-6:7077

In [ ]:
from pyspark.sql import SparkSession
from operator import add

spark_session = SparkSession.builder\
        .master("spark://yash-kukrejade-6:7077") \
        .appName("Lecture1_Example0_with_spark")\
        .config("spark.dynamicAllocation.enabled", True)\
        .config("spark.dynamicAllocation.shuffleTracking.enabled",True)\
        .config("spark.shuffle.service.enabled", False)\
        .config("spark.dynamicAllocation.executorIdleTimeout","30s")\
        .config("spark.executor.cores", 2)\
        .config("spark.driver.port",9999)\
        .config("spark.blockManager.port",10005)\
        .getOrCreate()

# RDD API
spark_context = spark_session.sparkContext

spark_context.setLogLevel("INFO")

In [ ]:
lines = spark_context.textFile('/home/ubuntu/i-have-a-dream.txt')
lines.take(5)